In [ ]:
import os

bucket = os.environ['WORKSPACE_BUCKET']


#from hail.plot import show
from pprint import pprint
from collections import Counter
from bokeh.plotting import output_file, save
import bokeh.io
from bokeh.io import *
from bokeh.resources import INLINE

In [ ]:
bokeh.io.output_notebook(INLINE) 
%matplotlib inline

In [ ]:
# Initialize Hail
import hail as hl
#import os
#from hail.plot import show

hl.init(default_reference='GRCh38')
#hl.plot.output_notebook()

In [ ]:
pheno = hl.import_table(f'{bucket}/data/gwas_v4/pheno_hail_final.tsv', key = "person_id",  impute=True,types={'person_id':hl.tstr})

In [ ]:
# read ancestry file using Hail
ancestry_pred = hl.import_table(f"{bucket}/data/ancestry_preds.tsv",
                               key="research_id", 
                               impute=True, 
                               types={"research_id":"tstr","pca_features":hl.tarray(hl.tfloat)})

In [ ]:
pheno=pheno.annotate(ancestry_pred=ancestry_pred[pheno.person_id])

In [ ]:
mt4=hl.read_matrix_table(f'{bucket}/data/gwas_v4/gwas_filtered.mt')
mt4.count()

In [ ]:
mt4 = mt4.annotate_cols(pheno = pheno[mt4.s])

In [ ]:
#modify this cell for each ancestry
anc='afr'

In [ ]:
mt4 = mt4.filter_cols(mt4.pheno.ancestry_pred.ancestry_pred==anc)

In [ ]:
covariates = [1.0, mt4.pheno.age, mt4.pheno.PC1, mt4.pheno.PC2, mt4.pheno.PC3, mt4.pheno.is_female]


In [ ]:
gwas = hl.logistic_regression_rows(y=mt4.pheno.label,
                                   test='wald',
                                 x=mt4.GT.n_alt_alleles(),
                                 covariates=covariates)
gwas.describe()

In [ ]:
gwas.export(f'{bucket}/data/multi_ancestry/gwas_{anc}.tsv')